In [ ]:

import os, shutil
import tensorflow.keras
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import array_to_img

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pydicom
from PIL import Image
import cv2
import shutil

In [6]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)
print("Num GPUs Available: ", len(tensorflow.config.list_physical_devices('GPU')))


Num GPUs Available:  1


C:\Users\Mojtaba Jafari Tadi\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\client\session.py:1751: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [7]:
#get working directory
os.getcwd()

'C:\\Users\\Mojtaba Jafari Tadi\\Projects_AMK\\Radiology Project'

In [8]:
os.listdir()

['Binary_Classification_Split',
 'data_DICOM',
 'fatpad',
 'reformatted_binary',
 'reformatted_binary_unscaled',
 'reformatted_data',
 'reformatted_data_unscaled',
 'xray_model.h5']

In [9]:
#change working directory to where the datasets are
root_directory=os.chdir('C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project')

In [10]:
#Provide train, validation, and test to tensorflow-keras ImageDataGenerator
base_dir_train = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train'
base_dir_validate = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/validation'
base_dir_test = 'C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/test'

print(base_dir_train)

C:/Users/Mojtaba Jafari Tadi/Projects_AMK/Radiology Project/Binary_Classification_Split/train


In [11]:
#run tensorflow-keras ImageDataGenerator on the directory which contains train, validate and test

# The validation data should not be augmented (just rescaled)

train_datagen = ImageDataGenerator(rescale=1./255,
                            samplewise_center=True, 
                              samplewise_std_normalization=True, 
                              horizontal_flip = True, 
                              vertical_flip = True, 
                              height_shift_range= 0.05, 
                              width_shift_range=0.1, 
                              rotation_range=15, 
                              shear_range = 0.1,
                              fill_mode = 'nearest',
                              zoom_range=0.1)


test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        base_dir_train,
        color_mode='rgb',
        target_size=(180, 180), 
        batch_size=9,
        shuffle=True,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        base_dir_validate, 
        color_mode='rgb',
        batch_size=9,
        shuffle=True,
        target_size=(180, 180), 
        class_mode='binary')

Found 3519 images belonging to 2 classes.
Found 1005 images belonging to 2 classes.


In [12]:
import matplotlib.pyplot as plt


def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

    

In [13]:
train_generator.image_shape
##batch = train_generator.next()
#image = batch[0].astype('float32')
#image.shape
#pyplot.imshow(image[8].reshape(180,180), cmap=pyplot.get_cmap('gray'))

(180, 180, 3)

In [14]:
from matplotlib import pyplot
gr=0
if gr: 
    fig, axs = plt.subplots(3,3, figsize=(15, 15), facecolor='w', edgecolor='k')
    axs = axs.ravel()
    for i in range(9):
        # define subplot
        # generate batch of images
        batch = train_generator.next()
        # convert to unsigned integers for viewing
        image = batch[0]
        # plot raw pixel data
        axs[i].imshow(image[i].reshape(180,180), cmap=pyplot.get_cmap('gray'))# show the figure


In [15]:
#load the root model and load trained weights into it
print(train_generator.n)
steps_per_epoch = int(train_generator.n/32)
print(steps_per_epoch)
validation_steps=int(validation_generator.n/32)
print(validation_steps)

3519
109
31


In [16]:
# should add smoothing to these plots
def plot_history(h):
    acc = h.history['acc']
    val_acc = h.history['val_acc']
    loss = h.history['loss']
    val_loss = h.history['val_loss']

    epochs = range(len(acc))

    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()

    plt.figure()

    plt.plot(epochs, loss, 'bo', label='Training loss')
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

    plt.show()

In [18]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing


def conv_block(filters, inputs):
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(inputs)
    x = layers.Conv2D(filters, 3, activation="relu", padding="same")(x)
    x = layers.BatchNormalization()(x)
    outputs = layers.MaxPool2D()(x)

    return outputs


def dense_block(units, dropout_rate, inputs):
    x = layers.Dense(units, activation="relu")(inputs)
    x = layers.BatchNormalization()(x)
    outputs = layers.Dropout(dropout_rate)(x)

    return outputs

In [19]:
def build_model(inputshape):
    inputs = keras.Input(inputshape)
    x = preprocessing.Rescaling(1.0 / 255)(inputs)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.Conv2D(16, 3, activation="relu", padding="same")(x)
    x = layers.MaxPool2D()(x)

    x = conv_block(32, x)
    x = conv_block(64, x)

    x = conv_block(128, x)
    x = layers.Dropout(0.2)(x)

    x = conv_block(256, x)
    x = layers.Dropout(0.2)(x)

    x = layers.Flatten()(x)
    x = dense_block(512, 0.7, x)
    x = dense_block(128, 0.5, x)
    x = dense_block(64, 0.3, x)

    outputs = layers.Dense(1, activation="sigmoid")(x)

    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


In [20]:
checkpoint_cb = tensorflow.keras.callbacks.ModelCheckpoint("xray_model.h5", save_best_only=True)

early_stopping_cb = tensorflow.keras.callbacks.EarlyStopping(
    patience=10, restore_best_weights=True
)

In [21]:
initial_learning_rate = 0.015
lr_schedule = tensorflow.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)

In [22]:
model = build_model(inputshape=train_generator.image_shape)
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 180, 180, 3)]     0         
_________________________________________________________________
rescaling (Rescaling)        (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 180, 180, 16)      448       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 180, 180, 16)      2320      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 90, 90, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 90, 90, 32)        4640      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 90, 90, 32)       

In [23]:
train_generator.class_indices


{'EF_cl0': 0, 'EF_cl1': 1}

In [26]:

METRICS = [
    tensorflow.keras.metrics.BinaryAccuracy(),
    tensorflow.keras.metrics.Precision(name="precision"),
    tensorflow.keras.metrics.Recall(name="recall"),
]
model.compile(
    optimizer=tensorflow.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss="binary_crossentropy",
    metrics=METRICS,
)


history = model.fit(
    train_generator,
    steps_per_epoch=int(train_generator.n/32),
    epochs=100,
    validation_data=validation_generator,
    validation_steps=int(validation_generator.n/32),
    callbacks=[checkpoint_cb, early_stopping_cb])

Epoch 1/100
109/109 [==============================] - 26s 241ms/step - loss: 0.7006 - binary_accuracy: 0.5148 - precision: 0.4713 - recall: 0.2668 - val_loss: 0.8579 - val_binary_accuracy: 0.4866 - val_precision: 0.4866 - val_recall: 1.0000
Epoch 2/100
109/109 [==============================] - 26s 238ms/step - loss: 0.6948 - binary_accuracy: 0.5250 - precision: 0.4984 - recall: 0.3269 - val_loss: 0.8006 - val_binary_accuracy: 0.4866 - val_precision: 0.4866 - val_recall: 1.0000
Epoch 3/100
109/109 [==============================] - 26s 235ms/step - loss: 0.7044 - binary_accuracy: 0.5005 - precision: 0.4841 - recall: 0.3828 - val_loss: 0.7241 - val_binary_accuracy: 0.4866 - val_precision: 0.4866 - val_recall: 1.0000
Epoch 4/100
109/109 [==============================] - 26s 238ms/step - loss: 0.7014 - binary_accuracy: 0.4893 - precision: 0.4587 - recall: 0.3165 - val_loss: 0.6945 - val_binary_accuracy: 0.5134 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 5/100
109/109 [===

In [ ]:
test_loss, test_acc = model.evaluate_generator(test_generator, steps=20)
test_acc